# Домашняя работа №5

## Задание 1. Написать теггер на данных с русским языком
1. Проверить UnigramTagger, BigramTagger, TrigramTagger и их комбинации
2. Написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
3. Сравнить все реализованные методы, сделать выводы
 
## Задание 2. Проверить, насколько хорошо работает NER. 
### Данные брать из http://www.labinform.ru/pub/named_entities/
1. Проверить NER из nltk/spacy/deeppavlov.
2. Написать свой NER, попробовать разные подходы.
  - передаём в сетку токен и его соседей.
  - передаём в сетку только токен.
  - свой вариант.
3. сравнить свои реализованные подходы на качество — вывести precision/recall/f1_score.

In [7]:
!pip install pyconll

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
import pyconll

In [2]:
!pip install wget
import wget
wget.download('https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-Taiga/master/ru_taiga-ud-train.conllu')
wget.download('https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-Taiga/master/ru_taiga-ud-dev.conllu')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=7472a97e43ae4dec61858b1a37e26aec20aeb9742485594ea77db387e6907b84
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


'ru_taiga-ud-dev.conllu'

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

from gensim.models import Word2Vec, FastText

from nltk.tag import DefaultTagger
from nltk.tag import UnigramTagger
from nltk.tag import BigramTagger, TrigramTagger
from nltk.tag import RegexpTagger
from nltk.corpus import names
import nltk
nltk.download('names')

[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Unzipping corpora/names.zip.


True

In [4]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

from scipy.sparse import hstack

In [5]:
import warnings
warnings.filterwarnings("ignore")

In [9]:
full_train = pyconll.load_from_file('ru_taiga-ud-train.conllu')
full_test = pyconll.load_from_file('ru_taiga-ud-dev.conllu')

In [10]:
for sent in full_train[:2]:
    for token in sent:
        print(token.form, token.upos)
    print()

Снова ADV
приобрел VERB
дозу NOUN
, PUNCT

В ADP
женщине NOUN
важна ADJ
верность NOUN
, PUNCT
а CCONJ
не PART
красота NOUN
. PUNCT



### 1.1 проверить UnigramTagger, BigramTagger, TrigramTagger и их комбмнации

In [11]:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form for token in sent])
    
    
MAX_SENT_LEN = max(len(sent) for sent in full_train)
MAX_ORIG_TOKEN_LEN = max(len(token.form) for sent in full_train for token in sent)
print('Наибольшая длина предложения', MAX_SENT_LEN)
print('Наибольшая длина токена', MAX_ORIG_TOKEN_LEN)

Наибольшая длина предложения 274
Наибольшая длина токена 161


In [12]:
default_tagger = nltk.DefaultTagger('NOUN')
default_tagger.evaluate(fdata_test)

0.24167987321711568

In [13]:
unigram_tagger = UnigramTagger(fdata_train)
unigram_tagger.evaluate(fdata_test)

0.6831418383518225

In [14]:
bigram_tagger = BigramTagger(fdata_train, backoff=unigram_tagger)
bigram_tagger.evaluate(fdata_test)

0.6859152139461173

In [15]:
trigram_tagger = TrigramTagger(fdata_train, backoff=bigram_tagger)
trigram_tagger.evaluate(fdata_test)

0.6867076069730587

In [16]:
def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff


backoff = DefaultTagger('NOUN') 
tag = backoff_tagger(fdata_train,  
                     [
                      UnigramTagger, 
                      BigramTagger, 
                      TrigramTagger
                     ],  
                     backoff = backoff) 
  
tag.evaluate(fdata_test)

0.785756735340729

## 1.2 написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов

In [18]:
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append(tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])
        
        
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label) 
test_enc_labels = le.transform(test_label)
le.classes_

array(['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM',
       'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X'],
      dtype='<U5')

In [53]:
for vectorizer in [CountVectorizer, HashingVectorizer, TfidfVectorizer]:

    scaler = StandardScaler(with_mean=False)
    coder = vectorizer(ngram_range=(1, 5), analyzer='char')
    

    X_train = coder.fit_transform(train_tok)
    X_test = coder.transform(test_tok)
    
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)    
    
    
    print(X_train.shape)
    lr = LogisticRegression(random_state=0, max_iter = 100, n_jobs=7)
    lr.fit(X_train, train_enc_labels)

    pred = lr.predict(X_test)

    print(vectorizer, accuracy_score(test_enc_labels, pred))

(176631, 93146)
<class 'sklearn.feature_extraction.text.CountVectorizer'> 0.8331022187004754
(176631, 1048576)
<class 'sklearn.feature_extraction.text.HashingVectorizer'> 0.8484548335974643
(176631, 93146)
<class 'sklearn.feature_extraction.text.TfidfVectorizer'> 0.8544968304278923


In [54]:
for vectorizer in [CountVectorizer, HashingVectorizer, TfidfVectorizer]:

    scaler = StandardScaler(with_mean=False)
    coder = vectorizer(ngram_range=(1, 5), analyzer='word')
    

    X_train = coder.fit_transform(train_tok)
    X_test = coder.transform(test_tok)
    
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)    
    
    
    print(X_train.shape)
    lr = LogisticRegression(random_state=0, max_iter = 100, n_jobs=7)
    lr.fit(X_train, train_enc_labels)

    pred = lr.predict(X_test)

    print(vectorizer, accuracy_score(test_enc_labels, pred))

(176631, 34451)
<class 'sklearn.feature_extraction.text.CountVectorizer'> 0.49554278922345485
(176631, 1048576)
<class 'sklearn.feature_extraction.text.HashingVectorizer'> 0.5009904912836767
(176631, 34451)
<class 'sklearn.feature_extraction.text.TfidfVectorizer'> 0.4948494453248811


In [55]:
scaler = StandardScaler(with_mean=False)
coder_1 = TfidfVectorizer(ngram_range=(1, 5), analyzer='char')
coder_2 = HashingVectorizer(ngram_range=(1, 5), analyzer='word')

X_train_1 = coder_1.fit_transform(train_tok)
X_test_1 = coder_1.transform(test_tok)

X_train_2 = coder_2.fit_transform(train_tok)
X_test_2 = coder_2.transform(test_tok)


X_train = hstack((X_train_1,X_train_2))
X_test = hstack((X_test_1,X_test_2))

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)    


print(X_train.shape)
lr = LogisticRegression(random_state=0, max_iter = 100, n_jobs=7)
lr.fit(X_train, train_enc_labels)

pred = lr.predict(X_test)

print('TfidfVectorizer_char + HashingVectorizer_word :', accuracy_score(test_enc_labels, pred))

(176631, 1141722)
TfidfVectorizer_char + HashingVectorizer_word : 0.856675911251981


__Выводы:__

Для nltk.tag лучший вариант это: Комбинация из DefaultTagger UnigramTagger BigramTagger TrigramTagger 0.785756735340729

Для Vectorizer лучший вариант это: Комбинация из LogisticRegression поверх 'TfidfVectorizer_char + HashingVectorizer_word' 0.856675911251981

## Задание 2. Проверить насколько хорошо работает NER
данные брать из http://www.labinform.ru/pub/named_entities/
1. взять нер из nltk
2. проверить deeppavlov
3. написать свой нер попробовать разные подходы:
 - передаём в сетку токен и его соседей
 - передаём в сетку только токен
4. сделать выводы по вашим экспериментам какой из подходов успешнее справляется


## 2.1 взять нер из nltk

In [20]:
!pip install corus

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 83 kB 2.0 MB/s 


In [21]:
import corus

In [22]:
from corus import load_ne5

In [23]:
wget.download('http://www.labinform.ru/pub/named_entities/collection5.zip')

'collection5.zip'

In [24]:
import zipfile
with zipfile.ZipFile('collection5.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [25]:
import nltk
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [26]:
!ls

Collection5	 ru_taiga-ud-dev.conllu    sample_data
collection5.zip  ru_taiga-ud-train.conllu


In [ ]:
# !unzip collection5.zip

In [27]:
from corus import load_ne5

In [28]:
def read_files(file_path):
    with open(file_path, encoding='utf8') as f:
        text = f.read()
        return text

In [29]:
records = load_ne5('Collection5/')
document = next(records)
document

Ne5Markup(
    id='last_35',
    text='Нового руководителя подмосковной прокуратуры нашли в столице\r\n\r\n\r\nИсполнять обязанности руководителя прокуратуры Московской области с сегодняшнего дня поручено Алексею Захарову, который до сих пор работал заместителем прокурора Москвы. На этом посту он заменит ушедшего в отставку Александра Аникина.\r\n\r\nПо данным газеты "Коммерсантъ", А.Аникин, который занял пост руководителя прокуратуры Московской области после знаменитого "игорного дела", за два года заменил 28 городских, районных и специализированных прокуроров, проведя внеплановую переаттестацию сотрудников.\r\n\r\nОднако проблемы возникли и у самого А.Аникина. По данным издания, у него были разногласия с двумя заместителями генерального прокурора - Виктором Гринем и Владимиром Малиновским. В результате первую попытку уйти в отставку А.Аникин предпринял еще весной, но его просьбу отклонил Юрий Чайка. Теперь же просьба прокурора Подмосковья удовлетворена. Он написал заявление и ушел в 

In [30]:
records

<generator object load_ne5 at 0x7f10f458a650>

In [32]:
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(document))) if hasattr(chunk, 'label') }

{('Tamarod', 'ORGANIZATION'),
 ('Абдель Азиз', 'PERSON'),
 ('Дауд', 'PERSON'),
 ('Египетская', 'PERSON'),
 ('Египта', 'PERSON'),
 ('Египте', 'PERSON'),
 ('Мохамед Абдель Азиз', 'PERSON'),
 ('Совета Шуры', 'PERSON'),
 ('Фронт', 'PERSON')}

## 2.2. проверить deeppavlov

In [ ]:
# !pip install deeppavlov

In [ ]:
# не пошла установка

## 2.3 написать свой нер попробовать разные подходы:
- передаём в сетку токен и его соседей
- передаём в сетку только токен

In [33]:
!pip install razdel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [34]:
from razdel import tokenize

In [36]:
records = corus.load_ne5('Collection5/')

In [37]:
words_docs = []
for ix, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):
       
        result = 'None'        
        
        for item in rec.spans:            
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'PER'):
                result = 'PER'
                break
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'ORG'):
                result = 'ORG'
                break
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'MEDIA'):
                result = 'MEDIA'
                break
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'LOC'):
                result = 'LOC'
                break
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'GEOPOLIT'):
                result = 'GEOPOLIT'
                break
                
    
        words.append([token.text, result])
    words_docs.extend(words)

In [38]:
df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])

In [39]:
df_words['tag'].value_counts()

None        219214
PER          21200
ORG          13651
LOC           4568
GEOPOLIT      4356
MEDIA         2482
Name: tag, dtype: int64

In [44]:
df_words.tail(10)

,word,tag
265461,и,ORG
265462,стабилизации,ORG
265463,Госдепа,ORG
265464,США,GEOPOLIT
265465,и,None
265466,работал,None
265467,послом,None
265468,на,None
265469,Украине,GEOPOLIT
265470,.,None


In [45]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, GlobalMaxPooling1D, Conv1D, GRU, LSTM, Dropout, Input
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [46]:
from sklearn import model_selection, preprocessing, linear_model

train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_words['word'], df_words['tag'])

# labelEncode целевую переменную
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [47]:
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y))
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))

train_data = train_data.batch(16)
valid_data = valid_data.batch(16)

In [48]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

In [49]:
def custom_standardization(input_data):
        return input_data

def data_prep(train_data, seq_len=1, vocab_size = 30000):    
    
    vocab_size = 30000
    #seq_len = 1

    vectorize_layer = TextVectorization(
        standardize=custom_standardization,
        max_tokens=vocab_size,
        output_mode='int',
        output_sequence_length=seq_len)


    # Make a text-only dataset (no labels) and call adapt to build the vocabulary.
    text_data = train_data.map(lambda x, y: x)
    vectorize_layer.adapt(text_data)
    return vectorize_layer

In [50]:
embedding_dim = 64

class modelNER(tf.keras.Model):
    def __init__(self):
        super(modelNER, self).__init__()
        self.emb = Embedding(vocab_size, embedding_dim)
        self.gPool = GlobalMaxPooling1D()
        self.fc1 = Dense(300, activation='relu')
        self.fc2 = Dense(50, activation='relu')
        self.fc3 = Dense(len(df_words['tag'].value_counts()), activation='softmax')

    def call(self, x):
        x = vectorize_layer(x)
        x = self.emb(x)
        pool_x = self.gPool(x)
        
        fc_x = self.fc1(pool_x)
        fc_x = self.fc2(fc_x)
        
        concat_x = tf.concat([pool_x, fc_x], axis=1)
        return self.fc3(concat_x)

In [51]:
vocab_size = 30000
vectorize_layer = data_prep(train_data, seq_len = 1, vocab_size = vocab_size)


mmodel = modelNER()
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
mmodel.fit(train_data, validation_data=valid_data, epochs=5)

Epoch 1/5
12444/12444 [==============================] - 62s 5ms/step - loss: 0.2822 - accuracy: 0.9178 - val_loss: 0.2112 - val_accuracy: 0.9384
Epoch 2/5
12444/12444 [==============================] - 59s 5ms/step - loss: 0.1205 - accuracy: 0.9638 - val_loss: 0.2194 - val_accuracy: 0.9395
Epoch 3/5
12444/12444 [==============================] - 57s 5ms/step - loss: 0.1064 - accuracy: 0.9660 - val_loss: 0.2278 - val_accuracy: 0.9400
Epoch 4/5
12444/12444 [==============================] - 58s 5ms/step - loss: 0.1018 - accuracy: 0.9667 - val_loss: 0.2728 - val_accuracy: 0.8813
Epoch 5/5
12444/12444 [==============================] - 58s 5ms/step - loss: 0.0993 - accuracy: 0.9673 - val_loss: 0.3008 - val_accuracy: 0.8766


In [52]:
vocab_size = 30000
vectorize_layer = data_prep(train_data, seq_len = 3, vocab_size = vocab_size)


mmodel = modelNER()
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
mmodel.fit(train_data, validation_data=valid_data, epochs=5)

Epoch 1/5
12444/12444 [==============================] - 57s 5ms/step - loss: 0.2912 - accuracy: 0.9145 - val_loss: 0.2113 - val_accuracy: 0.9370
Epoch 2/5
12444/12444 [==============================] - 57s 5ms/step - loss: 0.1225 - accuracy: 0.9631 - val_loss: 0.2128 - val_accuracy: 0.9396
Epoch 3/5
12444/12444 [==============================] - 59s 5ms/step - loss: 0.1078 - accuracy: 0.9658 - val_loss: 0.2137 - val_accuracy: 0.9406
Epoch 4/5
12444/12444 [==============================] - 57s 5ms/step - loss: 0.1027 - accuracy: 0.9667 - val_loss: 0.2218 - val_accuracy: 0.9398
Epoch 5/5
12444/12444 [==============================] - 59s 5ms/step - loss: 0.0997 - accuracy: 0.9674 - val_loss: 0.2368 - val_accuracy: 0.9391


#### Вывод.

Длина последовательности практически не влияет на результат